<link rel="preconnect" href="https://fonts.googleapis.com">
<link rel="preconnect" href="https://fonts.gstatic.com" crossorigin>
<link href="https://fonts.googleapis.com/css2?family=Titillium+Web:ital,wght@0,200;0,300;0,400;0,600;0,700;0,900;1,200;1,300;1,400;1,600;1,700&display=swap" rel="stylesheet">
<div class="box" style="padding: 10px; margin: 100px 90px; background-color: white; color: white; border-radius: 5px; font-size: 15px; box-shadow: rgba(0, 0, 0, 0.25) 0px 54px 55px, rgba(0, 0, 0, 0.12) 0px -12px 30px, rgba(0, 0, 0, 0.12) 0px 4px 6px, rgba(0, 0, 0, 0.17) 0px 12px 13px, rgba(0, 0, 0, 0.09) 0px -3px 5px;">
  <table style="padding: 10px; margin: auto auto;   border-radius: 20px; font-size: 15px;">
      <tr>
          <th colspan='2'><h1 style="text-align: center">
Natural Language Processing</br>
</h1>
<h2 style="text-align: center">
Course Assignment Two </br>
</h2>
</th>
<tr>
    <tr>
      <th colspan="2">Personal Info</th>
    </tr>
    <tr>
      <td>Ali</td>
    <td><b>Nikkhah</b></td>
    </tr>
    <tr>
      <td>Sarina</td>
    <td><b>Zahedi</b></td>
    </tr>
    <tr>
      <td>Ramtin</td>
    <td><b>Khoshnevis</b></td>
    </tr>
    <tr>
      <td>Github:</td>
      <td><a href="https://github.com/AliNikkhah2001/NaturalLanguageProcessing02" target="_blank">https://github.com/AliNikkhah2001/NaturalLanguageProcessing02</a></td>
    </tr>
  </table>
</div>


## Initializations

In [5]:
%%capture
!pip install transformers datasets torch
!pip install pandas tqdm
!pip install python-Levenshtein
!pip install jellyfish


In [3]:
# Import necessary libraries from Hugging Face and PyTorch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset
from tqdm.notebook import tqdm
import random
import torch
import pandas as pd
import os
import re
import Levenshtein
import pandas as pd

In [4]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Load of the Datasets

### word error dataset

In [34]:

# Define the paths to the datasets
faspell_main_path =  'faspell_main.txt'
faspell_ocr_path =  'faspell_ocr.txt'

# Read and clean the faspell_main dataset
faspell_main_df = pd.read_csv(faspell_main_path, sep='\t', encoding='utf-8', header=None, names=["misspelled", "corrected", "error-category"])
faspell_main_cleaned_df = faspell_main_df.rename(columns={"corrected": "correct", "misspelled": "wrong"})
faspell_main_cleaned_df = faspell_main_cleaned_df[['correct', 'wrong']]


# Read and clean the faspell_ocr dataset
faspell_ocr_df = pd.read_csv(faspell_ocr_path, sep='\t', encoding='utf-8', header=None, names=["misspelled", "corrected"])
faspell_ocr_cleaned_df = faspell_ocr_df.rename(columns={"corrected": "correct", "misspelled": "wrong"})
faspell_ocr_cleaned_df = faspell_ocr_cleaned_df[['correct', 'wrong']]

# Concatenate the cleaned dataframes
words_df = pd.concat([faspell_ocr_cleaned_df, faspell_ocr_cleaned_df], ignore_index=True)
words_df=words_df[1:]

# Print samples from the final DataFrame
print(words_df['correct'].sample(40))
print(words_df['wrong'].sample(40))

263          داشت
581          برای
842           این
1472        میرزا
1489        مداخل
871          اتاق
335         زیادی
322      دبیرستان
171       تابستان
785          چادر
451           قصه
186         تیراژ
1157          روز
342            زن
1064         داشت
935          برای
1256          بنا
449           فرش
233          حمید
1466        لیلای
904       انتخابي
604          نشان
1525      نستعلیق
1055        خواهد
1255          قصه
1533         کنیم
1598         کنار
4           آمدیم
902     انبارخانه
716            تا
1204        شوهرش
1545        هاشان
746          هایش
890           این
1340        دیپلم
1227        نداشت
1093      درخشنده
1474         مشکل
473      کامپیوتر
937         بردند
Name: correct, dtype: object
1193         سلى
483        كذشته
1526          نص
465      كارلران
1170          س!
1188        سروع
680        مببور
1393       لرملى
431        عذ ا!
1177         سام
1092       درأمد
621         لمان
1585         يهن
400          سهر
1306         

### Synthetic dataset generation of typo

In [7]:
# Load the list of Persian words
distinct_words_path = 'distinct_words.txt'
with open(distinct_words_path, 'r', encoding='utf-8') as file:
    words = file.read().splitlines()

# Functions to create typographical errors
def missing_char(word):
    pos = random.randint(0, len(word) - 1)
    return word[:pos] + word[pos + 1:]

def extra_char(word):
    pos = random.randint(0, len(word))
    char = random.choice('ابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی')
    return word[:pos] + char + word[pos:]

def swapped_chars(word):
    if len(word) < 5:
        return word
    pos1, pos2 = random.sample(range(len(word)), 2)
    word_list = list(word)
    word_list[pos1], word_list[pos2] = word_list[pos2], word_list[pos1]
    return ''.join(word_list)




In [8]:
additional_data = []
for word in tqdm(words, desc="Generating synthetic data"):
    if(len(word)>8):
        for _ in range(3):
            wordtype = random.choice([missing_char, extra_char, swapped_chars])
            gen_word = wordtype(word)
            additional_data.append({'correct': word, 'wrong': gen_word})

# Create a DataFrame from the additional data
additional_df = pd.DataFrame(additional_data)
# Print samples from the final DataFrame
print(additional_df['correct'].sample(40))
print(additional_df['wrong'].sample(40))
# Append the additional data to the original DataFrame
words_df = pd.concat([words_df, additional_df], ignore_index=True)
# Print samples from the final DataFrame
print(words_df['correct'].sample(40))
print(words_df['wrong'].sample(40))


Generating synthetic data:   0%|          | 0/453158 [00:00<?, ?it/s]

982689          لارینوف
686529           بهنمیر
784170           کوئرای
984139           كوزران
174123     كهكشان‌شناسى
844560          زیگزاگی
477016          پژوهانی
536685        خمیدگی‌ها
747599       امورفلاحتی
316932          بی‌ذاتم
760249    داکسی‌سایکلین
899644          بوئتیچر
860193     ادیاندانشگاه
261610         بایابانی
904104         شال‌هایی
5075           میانه‌رو
522877         گوشه‌است
724913           شین‌ها
704550      وسیاست‌مدار
747078        هدایت‌گری
744762       هیسپالنسیس
42094        دارالتالیف
710415         خوزستان‌
955231         مصاله‌ای
86897           پردازان
147840           نسازدت
406745       تله‌اسکرین
291951           وتهویه
307426      اندرانداختی
477314          تیمارون
778459          نردیکتر
113575        فرایند‌ها
494522           کنودسن
321286           اصباحی
904700         الاحتساب
381320       وکارگردانی
172337          لشکرشکن
926016       خلاصه‌ترین
377684         سی‌سی‌جی
525722         ارابه‌رو
Name: correct, dtype: object
237648     

## ADD HE-KASRE TO DF

In [35]:
import pandas as pd

# Load the annotated dataset
annotated_path = 'annotated.csv'
annotated_df = pd.read_csv(annotated_path, encoding='utf-8')
column_name = annotated_df.columns[0]

# Initialize a list to hold the rows for the new DataFrame
rows = []

# Function to process each row
def process_text(row):
    text = row[column_name]
    words = text.split()

    for word in words:
        # Handle `ه+` annotation
        if 'ه+' in word:
            wrong_word = word.replace('+', '')
            correct_word = word.replace('ه+', '')
            rows.append({'wrong': wrong_word, 'correct': correct_word})

        # Handle `-ه` annotation
        elif '-ه' in word:
            correct_word = word.replace('-ه', 'ه')
            wrong_word = word.replace('-ه', '')
            rows.append({'wrong': wrong_word, 'correct': correct_word})

# Apply the function to each row in the DataFrame
annotated_df.apply(process_text, axis=1)

# Create a new DataFrame from the rows list
hekasre_words_df = pd.DataFrame(rows, columns=['wrong', 'correct'])

# Append the additional data to the original DataFrame
words_df = pd.concat([words_df, hekasre_words_df], ignore_index=True)


In [36]:
dataset_size = words_df.size
print(f"Size of the dataset: {dataset_size}")

# Print number of columns
num_columns = words_df.shape[1]
print(f"Number of columns: {num_columns}")

# Print number of rows
num_rows = words_df.shape[0]
print(f"Number of rows: {num_rows}")


df_path = 'created_words_dataset.txt'
words_df.to_csv(df_path, sep='\t', index=False)

print(f"DataFrame written to created_words_dataset.txt")

Size of the dataset: 6920
Number of columns: 2
Number of rows: 3460
DataFrame written to created_words_dataset.txt


In [37]:
# Print samples from the final DataFrame
print(hekasre_words_df.head(40))
df_path = 'created_words_dataset_hekasre.txt'
hekasre_words_df.to_csv(df_path, sep='\t', index=False)

print(f"DataFrame written to created_words_dataset_hekasre.txt")

        wrong   correct
0      تعویضه     تعویض
1       پرچمه      پرچم
2      نیشکره     نیشکر
3       توسطه      توسط
4       مالکه      مالک
5       زمانه      زمان
6      نیشکره     نیشکر
7        ساله       سال
8      تعویضه     تعویض
9       پرچمه      پرچم
10    نموداره    نمودار
11     صعودیه     صعودی
12      سایشه      سایش
13      صدایه      صدای
14       نامه       نام
15       روزه       روز
16       یاده       یاد
17       بینه       بین
18  ایرانیانه  ایرانیان
19    دسترسیه    دسترسی
20      مردمه      مردم
21  اپلیکیشنه  اپلیکیشن
22     کادویه     کادوی
23     روابطه     روابط
24      تمامه      تمام
25   قرارداده   قرارداد
26       سطحه       سطح
27     اوایله     اوایل
28       سنگه       سنگ
29     مداحیه     مداحی
30       پسره       پسر
31      شهیده      شهید
32     مدافعه     مدافع
33     آلبومه     آلبوم
34     آموزشه     آموزش
35     سخنانه     سخنان
36   انتقادیه   انتقادی
37      صریحه      صریح
38      دکانه      دکان
39     دونبشه     دونبش
DataFrame writte

## Sentences Dataset

In [24]:
import zipfile
import os

# Define the path to the zip file and the extraction directory
zip_file_path = '/content/DataSet.zip'
extract_to_path = '/content/'

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("Extraction complete.")


Extraction complete.


In [25]:

real_word_path = 'DataSet/real-word'
real_word_files = ['gozar', 'plural', 'be', 'tanvin', 'hich', 'common']
real_word_files

['gozar', 'plural', 'be', 'tanvin', 'hich', 'common']

In [26]:

data = []

for file_name in real_word_files:
    correct_file = os.path.join(real_word_path, f'{file_name}/correct_{file_name}.txt')
    wrong_file = os.path.join(real_word_path, f'{file_name}/wrong_{file_name}.txt')

    with open(correct_file, 'r', encoding='utf-8') as f:
        correct_lines = f.readlines()

    with open(wrong_file, 'r', encoding='utf-8') as f:
        wrong_lines = f.readlines()

    for correct, wrong in zip(correct_lines, wrong_lines):
        data.append({'correct': correct.strip(), 'wrong': wrong.strip()})

sentences_df = pd.DataFrame(data)



In [27]:

print(sentences_df.sample(5).to_string)
dataset_size = sentences_df.size
print(f"Size of the dataset: {dataset_size}")

# Print number of columns
num_columns = sentences_df.shape[1]
print(f"Number of columns: {num_columns}")

# Print number of rows
num_rows = sentences_df.shape[0]
print(f"Number of rows: {num_rows}")




<bound method DataFrame.to_string of                                                 correct  \
1968  در شعاع وجودی انقلاب اسلامی و بنیانگذار آن حرک...   
2966  در هر صورت با هر چه بیشتر نزدیک شدن به انتخابا...   
4143           و در مساجدی شهر یا روستا حضور فعالی دارن   
413   به گزارش خبرگزاری فرانسه از پاریس در حالی که ن...   
5509  اعتمادبه نفس او در چارچوب و هنگام مهار توپ‌ها ...   

                                                  wrong  
1968  در شعاع وجودی انقلاب اسلامی وبنیانگزار آن حرکت...  
2966  در هر صورت با هر چه بیشتر نزدیک شدن به انتخابا...  
4143         و در مساجدهای شهر یا روستا حضور فعالی دارن  
413   به گزارش خبر گذاری فرانسه از پاریس در حالی که ...  
5509  اعتمادبنفس او در چارچوب و هنگام مهار توپ‌ها مث...  >
Size of the dataset: 15932
Number of columns: 2
Number of rows: 7966


In [28]:
# Load the dataset
dehkhoda_corpus = pd.read_csv('dehkhoda_corpus.csv')

# Assuming the dataset has columns 'wrong' and 'correct'
wrong = dehkhoda_corpus['correct']
correct = dehkhoda_corpus['wrong']

# Convert to DataFrame if needed
wrong_df = wrong.to_frame(name='wrong')
correct_df = correct.to_frame(name='correct')

# Combine the wrong and correct DataFrames into one
combined_df = pd.concat([wrong_df, correct_df], axis=1)

# Append to sentences_df
sentences_df = pd.concat([sentences_df, combined_df], ignore_index=True)
# Display the updated sentences_df
print(combined_df.sample().to_string())

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         wrong                                                                                                          

In [29]:

print(sentences_df.sample(5).to_string)
dataset_size = sentences_df.size
print(f"Size of the dataset: {dataset_size}")

# Print number of columns
num_columns = sentences_df.shape[1]
print(f"Number of columns: {num_columns}")

# Print number of rows
num_rows = sentences_df.shape[0]
print(f"Number of rows: {num_rows}")




<bound method DataFrame.to_string of                                                 correct  \
8642  من زیبا هستم من با همسرم در ایران زندگی میکنیم...   
2384  مرأسم دراویش قادریه هفته‌ای دو بار در روزهای ی...   
1407    همیشه افرادی مهم و تأثیرگذار در جوامعی بوده‌اند   
7174                     ناگاه آیینه بزرگ از طاق بیفتاد   
7021  یاری گرفتن از آگاهین و عالمان و التزام عملی به...   

                                                  wrong  
8642  من زیبا هستم من با همسرم در ایران زندگی میکنیم...  
2384  مراسم دراویش‌ها قادریه هفته‌ای دو بار در روزها...  
1407    همیشه افرادی مهم و ثأثیرگذار در جوامعی بوده‌اند  
7174                    ناگاه آیینه بزرگ از طاق بیافتاد  
7021  یاری گرفتن از آگاهان و عالمان و التزام عملی به...  >
Size of the dataset: 17348
Number of columns: 2
Number of rows: 8674


In [30]:
dataset_size = sentences_df.size
print(f"Size of the dataset: {dataset_size}")

# Print number of columns
num_columns = sentences_df.shape[1]
print(f"Number of columns: {num_columns}")

# Print number of rows
num_rows = sentences_df.shape[0]
print(f"Number of rows: {num_rows}")


df_path = 'created_sentences_dataset.txt'
sentences_df.to_csv(df_path, sep='\t', index=False)

print(f"DataFrame written to created_sentences_dataset.txt")

Size of the dataset: 17348
Number of columns: 2
Number of rows: 8674
DataFrame written to created_sentences_dataset.txt


In [48]:
annotated_path = 'annotated.csv'
annotated_df = pd.read_csv(annotated_path, encoding='utf-8')
annotated_df = pd.DataFrame(annotated_df)
# Extract the name of the single column
column_name = annotated_df.columns[0]

# Function to process each row
def process_text(row):
    text = row[column_name]
    correct_text = text.replace('ه+', '')
    wrong_text = text.replace('+', '')
    return pd.Series([correct_text, wrong_text], index=['correct', 'wrong'])

# Apply the function to each row and create a new DataFrame
hekasre = annotated_df.apply(process_text, axis=1)
print(hekasre.sample(5))
dataset_size = hekasre.size
print(f"Size of the dataset: {dataset_size}")

# Print number of columns
num_columns = hekasre.shape[1]
print(f"Number of columns: {num_columns}")

# Print number of rows
num_rows = hekasre.shape[0]
print(f"Number of rows: {num_rows}")



                                               correct  \
947  هر خانواده ی ایرانی یک به تو اعتماد دارم ولی ب...   
280  نوشته پسری که ماشین نداره چجوری به خودش اجازه ...   
230  انسان بیشرف و بی غیرت مثل خمینی عمر می کنند یک...   
416  میدونیدپایه اصلی ظهورکیان ماجوونا حالامیدونیدچ...   
904  امیرالمومنین به مالک اشتر می فرماید باید محبوب...   

                                                 wrong  
947  هر خانواده یه ایرانی یک به تو اعتماد دارم ولی ...  
280  نوشته پسری که ماشین نداره چجوری به خودش اجازه ...  
230  انسانه بیشرف و بی غیرت مثله خمینی عمر می کنند ...  
416  میدونیدپایه اصلیه ظهورکیان ماجوونا حالامیدونید...  
904  امیرالمومنین به مالکه اشتر می فرماید باید محبو...  
Size of the dataset: 1998
Number of columns: 2
Number of rows: 999


In [31]:
import shutil

# Define the source and destination file paths
source_file_path = '/content/created_sentences_dataset.txt'
destination_file_path = '/content/drive/My Drive/created_sentences_dataset.txt'

# Copy the file
shutil.copy(source_file_path, destination_file_path)

print("File copied to Google Drive successfully.")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/created_sentences_dataset.txt'

# Distance Model

In [61]:
import jellyfish

In [62]:
import Levenshtein
import jellyfish
import pandas as pd

class DistanceModel:
    def __init__(self, words_path):
        """
        Initializes the DistanceModel with a list of Persian words and a correction DataFrame.

        Args:
        words_path (str): Path to the file containing Persian words.
        words_df_path (str): Path to the file containing the correction DataFrame.
        """
        self.words = self.load_words(words_path)
        self.words_set = set(self.words)  # Use a set for O(1) average-time complexity lookups
        self.soundex_dict = self.build_soundex_dict(self.words)

    def load_words(self, path):
        """
        Loads words from a specified file.

        Args:
        path (str): Path to the file containing Persian words.

        Returns:
        list: List of words.
        """
        with open(path, 'r', encoding='utf-8') as file:
            words = file.read().splitlines()
        return words

    def load_words_df(self, path):
        """
        Loads the correction DataFrame from a specified file.

        Args:
        path (str): Path to the file containing the correction DataFrame.

        Returns:
        DataFrame: DataFrame containing 'wrong' and 'correct' columns.
        """
        return pd.read_csv(path, sep='\t')  # Assuming the file is tab-separated

    def build_soundex_dict(self, words):
        """
        Builds a dictionary mapping Soundex codes to lists of words.

        Args:
        words (list): List of words to build the Soundex dictionary.

        Returns:
        dict: Dictionary mapping Soundex codes to lists of words.
        """
        soundex_dict = {}
        for word in words:
            soundex_code = jellyfish.soundex(word)
            if soundex_code not in soundex_dict:
                soundex_dict[soundex_code] = []
            soundex_dict[soundex_code].append(word)
        return soundex_dict

    def calculate_distance(self, word1, word2):
        """
        Calculates the Levenshtein distance between two words.

        Args:
        word1 (str): The first word.
        word2 (str): The second word.

        Returns:
        int: The Levenshtein distance between the two words.
        """
        return Levenshtein.distance(word1, word2)

    def find_closest_word(self, word):
        """
        Finds the closest word in the loaded list of Persian words to the input word.

        Args:
        word (str): The input word to find the closest match for.

        Returns:
        str: The closest word from the list of Persian words.
        """
        soundex_code = jellyfish.soundex(word)
        candidate_words = self.soundex_dict.get(soundex_code, self.words)

        min_distance = float('inf')
        closest_word = None

        for persian_word in candidate_words:
            distance = self.calculate_distance(word, persian_word)
            if distance < min_distance:
                min_distance = distance
                closest_word = persian_word

        return closest_word

    def correct_words(self, words):
        """
        Corrects words in the input list using the words_df DataFrame.

        Args:
        words (list): List of words to be corrected.

        Returns:
        list: List of corrected words.
        """
        wrong_to_correct = dict(zip(self.words_df['wrong'], self.words_df['correct']))
        corrected_words = [wrong_to_correct.get(word, word) for word in words]
        return corrected_words

    def inference(self, sentence):
        """
        Processes the input sentence and returns a cleaned version with words replaced
        by their closest matches from the list of Persian words.

        Args:
        sentence (str): The input sentence to be cleaned.

        Returns:
        str: The cleaned sentence with words replaced by their closest matches.
        """
        # Split the sentence into words
        words = sentence.split()

        # Correct the words using the words_df DataFrame
        #words = self.correct_words(words)

        # Replace each word with its closest match from the list of Persian words
        cleaned_sentence = ' '.join(word if word in self.words_set else self.find_closest_word(word) for word in words)

        return cleaned_sentence



In [63]:
!ls

annotated.csv			   created_words_dataset.txt  dehkhoda_corpus.csv  faspell_ocr.txt
created_sentences_dataset.txt	   DataSet		      distinct_words.txt   __MACOSX
created_words_dataset_hekasre.txt  DataSet.zip		      faspell_main.txt	   sample_data


In [64]:

words_path = 'distinct_words.txt'

model = DistanceModel(words_path)


In [65]:
# Sample sentence for inference
sample_sentence = "منه بهه خیلیی میرویدی"
cleaned_sentence = model.inference(sample_sentence)
print(cleaned_sentence)
print()

منه به خیلی میروید



In [66]:
sample_sentences = [
    "هن به مدسه رفتم",
    "امروز هوا خیلیی خوب است",
    "من کتاب می‌خوامنم",
    "دیروز با دوستمم به پارک رفتم",
    "دانشگاها حوزاه اثر‌گدارری چه قبل از انیقلاب و چه بعد از انقلاب بودلند "
]

for sentence in sample_sentences:
    corrected_sentence = model.inference(sentence)
    print(f"Original: {sentence}")
    print(f"Corrected: {corrected_sentence}\n")


Original: هن به مدسه رفتم
Corrected: هن به مدسه رفتم

Original: امروز هوا خیلیی خوب است
Corrected: امروز هوا خیلی خوب است

Original: من کتاب می‌خوامنم
Corrected: من کتاب می‌خوانم

Original: دیروز با دوستمم به پارک رفتم
Corrected: دیروز با دوستم به پارک رفتم

Original: دانشگاها حوزاه اثر‌گدارری چه قبل از انیقلاب و چه بعد از انقلاب بودلند 
Corrected: دانشگاها حوزه اثرگذاری چه قبل از انقلاب و چه بعد از انقلاب بودند




#######################################################################################################################################################


# Model Training


### Special Tokenization

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from torch.utils.data import DataLoader
from accelerate import Accelerator, DataLoaderConfiguration
import os


# Disable wandb logging
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
from torch.utils.data import Dataset

class WordCorrectionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        input_text = self.dataframe.iloc[idx]["wrong"]
        target_text = self.dataframe.iloc[idx]["correct"]
        input_encoding = self.tokenizer(
            input_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            target_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        labels = target_encoding["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_encoding["input_ids"].flatten(),
            "attention_mask": input_encoding["attention_mask"].flatten(),
            "labels": labels.flatten(),
        }

    def print_samples(self, num_samples=5):
        """Prints a specified number of samples from the dataset."""
        print(f"{'Index':<10}{'Wrong Word':<20}{'Correct Word'}")
        print("="*50)
        for idx in range(min(num_samples, len(self.dataframe))):
            wrong = self.dataframe.iloc[idx]["wrong"]
            correct = self.dataframe.iloc[idx]["correct"]
            print(f"{idx:<10}{wrong:<20}{correct}")

class SentenceCorrectionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        input_text = self.dataframe.iloc[idx]["wrong"]
        target_text = self.dataframe.iloc[idx]["correct"]
        input_encoding = self.tokenizer(
            input_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            target_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        labels = target_encoding["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_encoding["input_ids"].flatten(),
            "attention_mask": input_encoding["attention_mask"].flatten(),
            "labels": labels.flatten(),
        }

    def print_samples(self, num_samples=5):
        """Prints a specified number of samples from the dataset."""
        print(f"{'Index':<10}{'Wrong Sentence':<50}{'Correct Sentence'}")
        print("="*100)
        for idx in range(min(num_samples, len(self.dataframe))):
            wrong = self.dataframe.iloc[idx]["wrong"]
            correct = self.dataframe.iloc[idx]["correct"]
            print(f"{idx:<10}{wrong:<50}{correct}")


In [ ]:
# Disable parallelism in tokenizers to avoid deadlocks
os.environ["TOKENIZERS_PARALLELISM"] = "false"


## GPT2-PERSIAN

In [ ]:
import difflib
def tag_errors(wrong_sentence, correct_sentence):
    wrong_words = wrong_sentence.split()
    correct_words = correct_sentence.split()

    matcher = difflib.SequenceMatcher(None, wrong_words, correct_words)
    tagged_wrong_sentence = ["<start>"]
    tagged_correct_sentence = ["<start>"]

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == 'equal':
            tagged_wrong_sentence.extend(wrong_words[i1:i2])
            tagged_correct_sentence.extend(correct_words[j1:j2])
        elif tag == 'replace':
            tagged_wrong_sentence.append(f"{' '.join(wrong_words[i1:i2])}")
            tagged_correct_sentence.append(f"<correct>{' '.join(correct_words[j1:j2])}</correct>")
        elif tag == 'delete':
            tagged_wrong_sentence.append(f"{' '.join(wrong_words[i1:i2])}")
        elif tag == 'insert':
            tagged_correct_sentence.append(f"<correct>{' '.join(correct_words[j1:j2])}</correct>")

    tagged_wrong_sentence.append("<end>")
    tagged_correct_sentence.append("<end>")

    return ' '.join(tagged_wrong_sentence), ' '.join(tagged_correct_sentence)

# Update the dataset with tagged sentences
tagged_data = sentences_df.apply(lambda row: tag_errors(row['wrong'], row['correct']), axis=1)
sentences_df['tagged_wrong'] = tagged_data.apply(lambda x: x[0])
sentences_df['tagged_correct'] = tagged_data.apply(lambda x: x[1])

In [ ]:
# Print some samples of the tagged dataframe
print(sentences_df[['tagged_wrong', 'tagged_correct']].head())

                                        tagged_wrong  \
0  <start> اما گروه دوم افزایش تدریجی قابل پیش‌بی...   
1  <start> دانشگاه‌ها حوزه اثر‌گزاری چه قبل از ان...   
2  <start> به‌اندازه ظرفیت خودش بر بازار اثر‌گزار...   
3  <start> که تشکیل دادسرای امور بین‌الملل برای ر...   
4  <start> در صورت نیاز نسبت به اصلاح نسبت به اثر...   

                                      tagged_correct  
0  <start> اما گروه دوم افزایش تدریجی قابل پیش‌بی...  
1  <start> دانشگاه‌ها حوزه <correct>اثرگذاری</cor...  
2  <start> به‌اندازه ظرفیت خودش بر بازار <correct...  
3  <start> که تشکیل دادسرای امور بین‌الملل برای ر...  
4  <start> در صورت نیاز نسبت به اصلاح نسبت به <co...  


In [ ]:
import torch

# Clear GPU cache
torch.cuda.empty_cache()

# Optionally, if you're running out of memory during training, you can also use this to clear cache within the training loop
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

In [ ]:
import torch

# Print GPU memory usage
print(f"Memory Allocated: {torch.cuda.memory_allocated()} bytes")
print(f"Memory Reserved: {torch.cuda.memory_reserved()} bytes")


Memory Allocated: 0 bytes
Memory Reserved: 0 bytes


In [ ]:
# Define Dataset class
class SentenceCorrectionDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        input_text = self.dataframe.iloc[idx]["tagged_wrong"]
        target_text = self.dataframe.iloc[idx]["tagged_correct"]
        input_encoding = self.tokenizer(
            input_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        target_encoding = self.tokenizer(
            target_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        labels = target_encoding["input_ids"].clone()
        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_encoding["attention_mask"].squeeze(),
            "labels": labels.squeeze(),
        }

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/gpt2-fa")

# Add special tokens to the tokenizer, including padding token
special_tokens_dict = {'additional_special_tokens': ['<start>', '<end>', '<correct>', '</correct>']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Initialize the dataset
sentence_dataset = SentenceCorrectionDataset(sentences_df, tokenizer)

# Function to print samples from the dataset
def print_tokenized_samples(dataset, num_samples=5):
    for i in range(num_samples):
        item = dataset[i]
        print(f"Sample {i+1}:")
        print(f"  Input IDs: {item['input_ids']}")
        print(f"  Attention Mask: {item['attention_mask']}")
        print(f"  Labels: {item['labels']}")
        print(f"  Input Text: {tokenizer.decode(item['input_ids'], skip_special_tokens=False)}")
        # Decode labels while handling the -100 values
        labels_decoded = [tokenizer.decode([token_id], skip_special_tokens=False) if token_id != -100 else '[PAD]' for token_id in item['labels'].tolist()]
        print(f"  Label Text: {' '.join(labels_decoded)}")
        print()

# Print some tokenized samples from the dataset
print_tokenized_samples(sentence_dataset, num_samples=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/808 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/875k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.75M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/104 [00:00<?, ?B/s]

Sample 1:
  Input IDs: tensor([42001,   492,   952,  1237,   803, 10414,  1045,   495,   285,  2391,
          293, 11197,   330,  1078,  1265,   285,  2850,  1302,   310,   285,
         3432,   231, 42002, 42005, 42005, 42005, 42005, 42005, 42005, 42005,
        42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005,
        42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005,
        42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005,
        42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005,
        42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005,
        42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005,
        42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005,
        42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005,
        42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005, 42005,
        42005, 42005, 42005, 42005, 42005

In [ ]:
model_name = "HooshvareLab/gpt2-fa"
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))


pytorch_model.bin:   0%|          | 0.00/485M [00:00<?, ?B/s]

Embedding(42006, 768)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=4,  # Reduce batch size to avoid memory issues
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,  # Accumulate gradients over multiple steps
    save_steps=1_000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_steps=500,
    report_to="none",
    fp16=True,  # Enable mixed precision training
    learning_rate=5e-4,
    dataloader_num_workers=2,
)

In [ ]:
# Disable parallelism in tokenizers to avoid deadlocks
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Train the model on sentence-level dataset

In [ ]:

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sentence_dataset,
    eval_dataset=sentence_dataset,  # Ideally, use a separate validation set
    tokenizer=tokenizer,
)

# Prepare dataloaders with Accelerator
accelerator = Accelerator()
train_loader = DataLoader(sentence_dataset, batch_size=training_args.per_device_train_batch_size, shuffle=True)
eval_loader = DataLoader(sentence_dataset, batch_size=training_args.per_device_eval_batch_size)

model, train_loader, eval_loader = accelerator.prepare(
    model, train_loader, eval_loader
)

# Train the model on sentence-level dataset
trainer.train()


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss
1000,4.133000,3.212128


In [ ]:
# Save the model
model_save_path = "./fine_tuned_gpt_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

# Final Mixed Model

# Final Model and i-o GPT

In [ ]:
%%capture
!pip install transformers datasets torch
!pip install pandas tqdm
!pip install python-Levenshtein
!pip install jellyfish


In [42]:
import pandas as pd
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import Levenshtein
import jellyfish

class DistanceModel:
    def __init__(self, words_path):
        self.words = self.load_words(words_path)
        self.words_set = set(self.words)
        self.soundex_dict = self.build_soundex_dict(self.words)

    def load_words(self, path):
        with open(path, 'r', encoding='utf-8') as file:
            words = file.read().splitlines()
        return words

    def build_soundex_dict(self, words):
        soundex_dict = {}
        for word in words:
            soundex_code = jellyfish.soundex(word)
            if soundex_code not in soundex_dict:
                soundex_dict[soundex_code] = []
            soundex_dict[soundex_code].append(word)
        return soundex_dict

    def calculate_distance(self, word1, word2):
        return Levenshtein.distance(word1, word2)

    def find_closest_word(self, word):
        soundex_code = jellyfish.soundex(word)
        candidate_words = self.soundex_dict.get(soundex_code, self.words)
        min_distance = float('inf')
        closest_word = None
        for persian_word in candidate_words:
            distance = self.calculate_distance(word, persian_word)
            if distance < min_distance:
                min_distance = distance
                closest_word = persian_word
        return closest_word

    def inference(self, sentence):
        words = sentence.split()
        cleaned_sentence = ' '.join(word if word in self.words_set else self.find_closest_word(word) for word in words)
        return cleaned_sentence

class RegexModel:
    def __init__(self, words_df_path):
        self.words_df = pd.read_csv(words_df_path, encoding='utf-8', sep='\t')
        self.wrong_to_correct = dict(zip(self.words_df['wrong'], self.words_df['correct']))
        self.regex = self.compile_regex()

    def compile_regex(self):
        pattern = '|'.join(r'\b' + re.escape(wrong) + r'\b' for wrong in self.wrong_to_correct.keys())
        return re.compile(pattern)

    def correct_text(self, text):
        def replace(match):
            return self.wrong_to_correct[match.group(0)]
        return self.regex.sub(replace, text)

class CombinedModel:
    def __init__(self, model_path, words_path, words_df_path, use_gpt2=False):
        self.use_gpt2 = use_gpt2
        if self.use_gpt2:
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.model = AutoModelForCausalLM.from_pretrained(model_path)
            self.model.resize_token_embeddings(len(self.tokenizer))
        self.distance_model = DistanceModel(words_path)
        self.regex_model = RegexModel(words_df_path)

    def inference(self, sentence):


        # Second pass through the RegexModel
        cleaned_sentence = self.distance_model.inference(sentence)
        corrected_sentence = self.regex_model.correct_text(cleaned_sentence)


        if self.use_gpt2:
            input_ids = self.tokenizer.encode(f"<start> {corrected_sentence} <end>", return_tensors='pt')
            with torch.no_grad():
                outputs = self.model.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
            final_output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            final_output_words = final_output.split()
        else:
            final_output_words = corrected_sentence.split()

        input_words = sentence.split()
        if len(final_output_words) != len(input_words):
            final_output_words = final_output_words[:len(input_words)]
            if len(final_output_words) < len(input_words):
                final_output_words += input_words[len(final_output_words):]

        return ' '.join(final_output_words)

In [43]:

# Example usage
words_path = "distinct_words.txt"
words_df_path = "created_words_dataset.txt"
combined_model = CombinedModel(model_path=None, words_path=words_path, words_df_path=words_df_path, use_gpt2=False)




In [44]:
# Test inference
test_sentence = " 	به مناسبته روزه جهانیه ترجمه البته با یک روز تاخیر"
result = combined_model.inference(test_sentence)
print(result)

به مناسبت روز جهانی ترجمه البته با یک روز تاخیر


In [45]:
# Test inference
test_sentence = " دانشگاها حوزاه اثر‌گدارری چه قبل از انیقلاب و چه بعد از انقلاب بودلند "
result = combined_model.inference(test_sentence)
print(result)

دانشگاها حوزه اثرگذاری چه قبل از انقلاب و چه بعد از انقلاب بودند


In [46]:
import random

# Assuming you have a sentences_df DataFrame containing the test sentences

# Generate 20 random indices for sampling
random_indices = random.sample(range(len(sentences_df)), 20)

for idx in random_indices:
    test_sentence = sentences_df.iloc[idx]['wrong']
    expected_output = sentences_df.iloc[idx]['correct']

    # Use the combined_model to get the model output
    model_output = combined_model.inference(test_sentence)

    print("Test Sentence:", test_sentence)
    print("Expected Output:", expected_output)
    print("Model Output:", model_output)
    print("\n")

Test Sentence: هم‌گرایی برای بانجام‌رساندن آنها وجود داشته_باشد
Expected Output: هم‌گرایی برای به انجام‌رساندن آنها وجود داشته_باشد
Model Output: هم‌گرایی برای به‌انجام‌رساند آنها وجود داشته‌باشد


Test Sentence: بنابراین ادامه این روند به نفع صنعت بیمه و حتی بیمهبیمه‌گذاران در آینده نخواهد_بود
Expected Output: بنابراین ادامه این روند به نفع صنعت بیمه و حتی بیمه‌گذاران در آینده نخواهد_بود
Model Output: بنابراین ادامه این روند به نفع صنعت بیمه و حتی بیمه‌گذاران در آینده نخواهدبود


Test Sentence: و دارنده این اسما عجایب‌ها از او مشاهده گردد
Expected Output: و دارنده این اسما عجایب از او مشاهده گردد
Model Output: و دارنده این اسما عجایب‌ها از این مشاهده گردد


Test Sentence: 3. به ستایش ایرانیان باستان می‌پردازد و درباره آداب و رسوم ایرانیان باستان سخن می‌گوید دفتر پارسی نامه در مجموع شامل دو شعر است که گوته در پانزدهم مارس 1815 نوشته‎است.
Expected Output: 3. به ستایش ایرانیان باستان می‌پردازد و درباره آداب و رسوم ایرانیان باستان سخن می‌گوید دفتر پارسی نامه در مجموع شامل دو شهر است که گو

In [50]:
import random


random_indices = random.sample(range(len(sentences_df)), 20)

for idx in random_indices:
    test_sentence = sentences_df.iloc[idx]['wrong']
    expected_output = sentences_df.iloc[idx]['correct']

    # Use the combined_model to get the model output
    model_output = combined_model.inference(test_sentence)

    print("Test Sentence:", test_sentence)
    print("Expected Output:", expected_output)
    print("Model Output:", model_output)
    print("\n")

Test Sentence: و اصول هارمونی و آهنگسازی را زیر نظر محمد حق‌گو فرا گرفت
Expected Output: و اصولرمونی و آهنگسازی را زیر نظر محمد حق‌گو فرا گرفت
Model Output: و اصول هارمونی و آهنگسازی را زیر نظر محمد حق‌گو فرا گرفت


Test Sentence: هزینه‌های مراسم و کلاس‌ها از محل موقوفات و نذوراتات تقبل می‌شود
Expected Output: هزینه‌های مرأسم و کلاس‌ها از محل موقوفات و نذورات تقبل می‌شود
Model Output: هزینه‌های مراسم و کلاس‌ها از محل موقوفات و نذوراتی تقبل می‌شود


Test Sentence: که تشفعون فی الحوایج اذا امتنعت من طلابها مردم در حاجت‌هایشان و جاهایی که محتاج می‌شوند
Expected Output: که تشفعون فی الحوائج اذا امتنعت من طلاب مردم در حاجت‌هایشان و جاهایی که محتاج می‌شوند
Model Output: که تیفون فی الحوایج اذا امتناع من طلبها مردم در حاجت‌های و جاهایی که محتاج می‌بشوند


Test Sentence: بروز آسیب‌های عروقی و عصبی و فشارهایی که توده به اعضاهای مجاور بدن وارد می‌کند
Expected Output: بروز آسیب‌های عروقی و عصبی و فشارهایی که توده به اعضای مجاور بدن وارد می‌کند
Model Output: بروز آسیب‌های عروقی و عصبی و فشارهایی ک